<a href="https://colab.research.google.com/github/hwankang/Pytorch_tutorials/blob/main/hwankang0816_50_Adam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
def v1(beta2, t, g):
    V = 0
    for i in range(t):
        V = beta2 * V + (1-beta2) * g**2
    return V

def v2(beta2, t, g):
    return (1 - beta2) * np.sum([beta2**(t-1-i)*g**2 for i in range(t)])

print(v1(0.9, 10, 0.1))
print(v2(0.9, 10, 0.1))

In [ ]:


%pylab inline
import numpy as np
import pandas as pd

from sklearn.model_selection import  train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_iris

In [ ]:
import matplotlib.pyplot  as plt
iris = load_iris()

# setosa_x = iris.data[:50]
# setosa_y = iris.target[:50]
# versicolor_x = iris.data[50:100]
# versicolor_y = iris.target[50:100]
# scatter(setosa_x[:, 0], setosa_x[:, 2])
# scatter(versicolor_x[:, 0], versicolor_x[:, 2])

# Extract sepal length, petal length from Setosa and Versicolor
data = iris.data[:100, [0, 2]]

# Standardization
scaler = StandardScaler()
data = scaler.fit_transform(data)


# Split data to test and train data
train_x, test_x, train_y, test_y = train_test_split(data, iris.target[:100], test_size=0.3)

# Plotting data
plt.figure(1)
scatter(data[:50, 0], data[:50, 1], label='Setosa')
scatter(data[50:100, 0], data[50:100, 1], label='Versicolour')
title('Iris Data')
xlabel('sepal length')
ylabel('petal length')
grid()
legend()


#SGD with Adam
w =   np.array([ 0.09370901, -0.24480254, -0.84210235]) # np.random.randn(2 + 1)

def predict(w, x):
    N = len(x)
    yhat = w[1:].dot(x.T) + w[0]
    return yhat

def adam_nn(w, X, Y, eta=0.001, alpha=0.001, beta1=0.9, beta2=0.999, weight_size=2):
    """
    @param eta <float>: learning rate
    @param alpha <float>: Step Size (suggested default is 0.001)
    """
    N = len(X)
    e = 1e-8
    T = 10000
    M = np.zeros(weight_size + 1) # First moment estimate
    V = np.zeros(weight_size + 1) # Second moment estimate

    for t in range(N):
        x = X[t]
        y = Y[t]
        x = x.reshape((-1, 2))
        yhat = predict(w, x)

        # Calculate the gradients
        gradient_w = 2/N*-(y-yhat).dot(x)
        gradient_b = 2/N*-(y-yhat)

        # Update biased first moment estimate
        M[1:] = beta1 * M[1:] + (1-beta1) * gradient_w
        M[0]  = beta1 * M[0]  + (1-beta1) * gradient_b

        # Update biased second raw moment estimate
        V[1:] = beta2 * V[1:] + (1-beta2) * gradient_w**2
        V[0]  = beta2 * V[0]  + (1-beta2) * gradient_b**2

        # Compute bias-corrected first moment estimate
        Mw = M[1:]/(1-beta1**t + e)
        Mb = M[0]/(1-beta1**t + e)

        # Compute bias-corrected second raw moment estimate
        Vw = V[1:]/(1-beta2**t)
        Vb = V[0]/(1-beta2**t)

        # Compute Deltas
        delta_w = alpha * Mw/(np.sqrt(Vw) + e)
        delta_b = alpha * Mb/(np.sqrt(Vb) + e)

#         delta_w = alpha * M[1:]/(np.sqrt(V[1:]) + e)
#         delta_b = alpha * M[0]/(np.sqrt(V[0]) + e)

        w[1:] = w[1:] - delta_w
        w[0] = w[0] - delta_b

    return w

accu_ar=[]
for i in range(50):
    w = adam_nn(w, train_x, train_y)

    # Accuracy Test
    yhats = predict(w, test_x)
    yhats = np.where(yhats >= 0.5, 1, 0)
    accuracy = round(accuracy_score(test_y, yhats), 2)
    accu_ar.append(accuracy)
    print(f'[{i:2}] Accuracy: {accuracy:<4.2}')
plt.figure(2)
plt.title("Adam")
plt.xlabel("Step")
plt.ylabel("Accuracy")
plt,plot(accu_ar, color='blue',label='Adam', marker='o')
plt.legend(loc='lower righr')
grid()
plt.show()